#  Segmenting and Clustering Project I.Guerrero

<font size = 5> Question 1

<font size = 2>For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

>Start by creating a new Notebook for this assignment.

>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

>3. To create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests
import lxml.html as lh

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
#let’s parse the first row as our header
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [5]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        


In [6]:
#Just to be sure, let’s check the length of each column.
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
#Replace al values of "\n" 
df = df.replace('\n','', regex=True)
df.rename(columns = {'Postal Code\n':'Postal Code','Borough\n':'Borough', 'Neighborhood\n': 'Neighborhood'}, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#Drop the codes that are not yet assigned
df.replace('Not assigned',np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#Exploring the tast items of the DF
df.tail()

,Postal Code,Borough,Neighborhood
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
180,,Canadian postal codes,


In [12]:
#Reset the index and drop the last item since it is not useful
df.reset_index(inplace=True)
df.drop(['index'], axis=1 , inplace=True)

last_item = len(df) - 1
df.drop([last_item], inplace=True)

df.tail()

,Postal Code,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [13]:
print(df.shape)


(103, 3)


<font size = 5> Question 2
    
<font size = 2>Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this

In [14]:
#Here I tried to use the geopy library
'''
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


#address = {"query": {'postalCode': 'M8X', 'countryRegion': 'Canada', 'locality': 'Toronto', 'adminDistrict': 'Ontario'}}
address = "Canada, North York, Parkwoods"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
'''

'\nfrom geopy.geocoders import Nominatim # convert an address into latitude and longitude values\n\n\n#address = {"query": {\'postalCode\': \'M8X\', \'countryRegion\': \'Canada\', \'locality\': \'Toronto\', \'adminDistrict\': \'Ontario\'}}\naddress = "Canada, North York, Parkwoods"\ngeolocator = Nominatim(user_agent="toronto_explorer")\nlocation = geolocator.geocode(address)\nlatitude = location.latitude\nlongitude = location.longitude\nprint(\'The geograpical coordinate of Toronto are {}, {}.\'.format(latitude, longitude))\n'

In [15]:
'''
i = 0
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
lat_lng_coords = pd.DataFrame(columns=column_names)

for borough, neighborhood in zip(df['Borough'], df['Neighborhood']):
    first_neighborhood = neighborhood.split(",")
    neighborhood = first_neighborhood[0].replace("'", " ")

    address = borough +", "+ neighborhood

    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    i+=1
    print(i)
    lat_lng_coords = lat_lng_coords.append({'Latitude': latitude,
                                            'Longitude': longitude}, ignore_index=True)
    if i>20:
        break
 '''

'\ni = 0\ncolumn_names = [\'Latitude\', \'Longitude\'] \n\n# instantiate the dataframe\nlat_lng_coords = pd.DataFrame(columns=column_names)\n\nfor borough, neighborhood in zip(df[\'Borough\'], df[\'Neighborhood\']):\n    first_neighborhood = neighborhood.split(",")\n    neighborhood = first_neighborhood[0].replace("\'", " ")\n\n    address = borough +", "+ neighborhood\n\n    geolocator = Nominatim(user_agent="toronto_explorer")\n    location = geolocator.geocode(address)\n    latitude = location.latitude\n    longitude = location.longitude\n    i+=1\n    print(i)\n    lat_lng_coords = lat_lng_coords.append({\'Latitude\': latitude,\n                                            \'Longitude\': longitude}, ignore_index=True)\n    if i>20:\n        break\n '

In [16]:
#Here I decided to use the .csv file
#Import data set
df_latlon = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlon.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#Sort data frame of borough and Neighborhood
df_sorted = df.sort_values('Postal Code'). reset_index(drop=True)
#Sort data frame of Lat. and Lon.
df_latlon_sorted = df_latlon.sort_values('Postal Code').reset_index(drop=True)
#Join both frames and check in order is the same
df_complete = df_sorted.join(df_latlon_sorted, rsuffix = ' check order', sort = True)       
df_complete.head()


,Postal Code,Borough,Neighborhood,Postal Code check order,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [18]:
#Drop unwanted columns and print the finished data frame
df_complete = df_complete.drop('Postal Code check order', axis=1)
df_complete.head(12)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<font size = 5> Question 3
    
<font size = 2>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

>Just make sure:

>1.to add enough Markdown cells to explain what you decided to do and to report any observations you make.

>2.to generate maps to visualize your neighborhoods and how they cluster together.

In [19]:
#Use geopy library to get the latitude and longitude values of
from geopy.geocoders import Nominatim

address = "Toronto, Ontario"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude={}, Longitude={}".format(latitude, longitude))

Latitude=43.6534817, Longitude=-79.3839347


In [20]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Borough'], df_complete['Neighborhood']):
    label = '({}) / {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [21]:
# Create a data frame with only boroughs that contain the word Toronto
data_list=[]
df_toronto = pd.DataFrame(data_list)

i=0
for borough in df_complete['Borough']:
    isinit = borough.find('Toronto')
    if isinit != -1:
        df_toronto = df_toronto.append(df_complete.loc[i])
    i = i+1

df_toronto.reset_index(drop=True)


,Borough,Latitude,Longitude,Neighborhood,Postal Code
0,East Toronto,43.676357,-79.293031,The Beaches,M4E
1,East Toronto,43.679557,-79.352188,"The Danforth West, Riverdale",M4K
2,East Toronto,43.668999,-79.315572,"India Bazaar, The Beaches West",M4L
3,East Toronto,43.659526,-79.340923,Studio District,M4M
4,Central Toronto,43.728020,-79.388790,Lawrence Park,M4N
5,Central Toronto,43.712751,-79.390197,Davisville North,M4P
6,Central Toronto,43.715383,-79.405678,North Toronto West,M4R
7,Central Toronto,43.704324,-79.388790,Davisville,M4S
8,Central Toronto,43.689574,-79.383160,"Moore Park, Summerhill East",M4T
9,Central Toronto,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest...",M4V


In [22]:
# createa second map of Toronto using latitude and longitude values
map_toronto_2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map with only boroughs that contain the word Toronto
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '({}) / {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_2)

map_toronto_2

In [23]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'F52OIEVBKQK3TDLU1OBLIYE4I2AO22A5XIWTM2Y4XBFTYAMC' 
CLIENT_SECRET = '1XDIO2E2ENSJSOS0TVRQTZLXA1X42X1DUMD5AHTK1EY2GPXJ'
VERSION = '20180605'

In [24]:
#Define the limit of responses and radius of search
LIMIT = 100
radius = 500
#url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)


In [25]:
#Function to explore and get nearby venues for a data frame of neighborhoods 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [27]:
print(toronto_venues.shape)
toronto_venues.loc[250:260]

(1605, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
250,Church and Wellesley,43.66586,-79.38316,Barbara Hall Park,43.666879,-79.381068,Park
251,Church and Wellesley,43.66586,-79.38316,Sansotei Ramen 三草亭,43.666735,-79.385353,Ramen Restaurant
252,Church and Wellesley,43.66586,-79.38316,Dal Moro's Fresh Pasta To Go,43.666641,-79.385400,Italian Restaurant
253,Church and Wellesley,43.66586,-79.38316,Hero Certified Burgers,43.665624,-79.380904,Burger Joint
254,Church and Wellesley,43.66586,-79.38316,Ho's Team Barber Shop,43.665630,-79.381359,Salon / Barbershop
255,Church and Wellesley,43.66586,-79.38316,T-Swirl Crepe,43.663452,-79.384125,Creperie
256,Church and Wellesley,43.66586,-79.38316,Glad Day Bookshop,43.665271,-79.380785,Bookstore
257,Church and Wellesley,43.66586,-79.38316,FUEL+,43.664399,-79.380427,Juice Bar
258,Church and Wellesley,43.66586,-79.38316,Si Lom,43.665010,-79.380683,Thai Restaurant
259,Church and Wellesley,43.66586,-79.38316,Coach House Restaurant,43.664991,-79.384814,Diner


In [28]:
#how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


In [29]:
#Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot =toronto_onehot.join(toronto_venues['Neighborhood'],lsuffix='_caller')

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.shape

(1605, 229)


(1605, 229)

In [30]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 229)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_caller,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.0000,0.017857,0.000000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.00000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.017857,0.053571,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.017857,0.000000,0.017857,0.000000,0.00,0.017857,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.000000,0

In [31]:
#print each neighborhood along with the top 5 most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Burrito Place
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant


In [32]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2])

In [33]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.shape # check the last columns!

(39, 11)

In [34]:
toronto_merged.reset_index(drop = True).head()

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,43.676357,-79.293031,The Beaches,M4E,2,Park,Trail,Pub,Neighborhood_caller,Health Food Store
1,East Toronto,43.679557,-79.352188,"The Danforth West, Riverdale",M4K,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
2,East Toronto,43.668999,-79.315572,"India Bazaar, The Beaches West",M4L,2,Fast Food Restaurant,Pub,Steakhouse,Sushi Restaurant,Ice Cream Shop
3,East Toronto,43.659526,-79.340923,Studio District,M4M,2,Café,Coffee Shop,Gastropub,Bakery,Brewery
4,Central Toronto,43.728020,-79.388790,Lawrence Park,M4N,4,Park,Swim School,Bus Line,Event Space,Ethiopian Restaurant


In [35]:
#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<font size = 5> Examine the clusters

In [40]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0].reset_index(drop=True)

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.696948,-79.411307,Forest Hill North & West,M5P,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio
1,Downtown Toronto,43.662696,-79.400049,"University of Toronto, Harbord",M5S,0,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore
2,Downtown Toronto,43.669542,-79.422564,Christie,M6G,0,Grocery Store,Café,Park,Candy Store,Restaurant
3,West Toronto,43.669005,-79.442259,"Dufferin, Dovercourt Village",M6H,0,Bakery,Pharmacy,Art Gallery,Furniture / Home Store,Supermarket
4,West Toronto,43.661608,-79.464763,"High Park, The Junction South",M6P,0,Thai Restaurant,Café,Mexican Restaurant,Flea Market,Cajun / Creole Restaurant


In [41]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1].reset_index(drop=True)

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.711695,-79.416936,Roselawn,M5N,1,Garden,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store


In [42]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2].reset_index(drop=True)

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,43.676357,-79.293031,The Beaches,M4E,2,Park,Trail,Pub,Neighborhood_caller,Health Food Store
1,East Toronto,43.679557,-79.352188,"The Danforth West, Riverdale",M4K,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
2,East Toronto,43.668999,-79.315572,"India Bazaar, The Beaches West",M4L,2,Fast Food Restaurant,Pub,Steakhouse,Sushi Restaurant,Ice Cream Shop
3,East Toronto,43.659526,-79.340923,Studio District,M4M,2,Café,Coffee Shop,Gastropub,Bakery,Brewery
4,Central Toronto,43.712751,-79.390197,Davisville North,M4P,2,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop
5,Central Toronto,43.715383,-79.405678,North Toronto West,M4R,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Seafood Restaurant,Salon / Barbershop
6,Central Toronto,43.704324,-79.388790,Davisville,M4S,2,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant
7,Central Toronto,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest...",M4V,2,Coffee Shop,Pub,Bagel Shop,Bank,Vietnamese Restaurant
8,Downtown Toronto,43.667967,-79.367675,"St. James Town, Cabbagetown",M4X,2,Coffee Shop,Restaurant,Café,Pizza Place,Bakery
9,Downtown Toronto,43.665860,-79.383160,Church and Wellesley,M4Y,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub


In [43]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3].reset_index(drop=True)

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.689574,-79.38316,"Moore Park, Summerhill East",M4T,3,Gym,Playground,Yoga Studio,Dance Studio,Electronics Store


In [44]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4].reset_index(drop=True)

,Borough,Latitude,Longitude,Neighborhood,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.728020,-79.388790,Lawrence Park,M4N,4,Park,Swim School,Bus Line,Event Space,Ethiopian Restaurant
1,Downtown Toronto,43.679563,-79.377529,Rosedale,M4W,4,Park,Playground,Trail,Cupcake Shop,Electronics Store
